# Create AMITT website sql

Creates the sqlite file used as a datasource in the AMITT website tests. 

In [1]:
# Get AMITT variables

import pandas as pd
import sqlite3 as sql
from generate_amitt_ttps import Amitt
pd.set_option('display.max_rows', 1000)


# Generate AMITT datasets
amitt = Amitt()

# Check which amitt variables we can see from here
print('{}'.format(vars(amitt).keys()))
vars(amitt)['cross_counterid_techniqueid']

dict_keys(['df_phases', 'df_frameworks', 'df_techniques', 'df_tasks', 'df_incidents', 'df_counters', 'df_detections', 'df_actortypes', 'df_resources', 'df_responsetypes', 'df_metatechniques', 'it', 'df_tactics', 'df_techniques_per_tactic', 'df_counters_per_tactic', 'phases', 'tactics', 'techniques', 'counters', 'metatechniques', 'actortypes', 'resources', 'num_tactics', 'cross_counterid_techniqueid', 'cross_counterid_resourceid', 'cross_counterid_actortypeid'])


,amitt_id,technique_id
0,C00006,T0007
0,C00006,T0015
0,C00006,T0018
0,C00006,T0043
0,C00006,T0053
0,C00006,T0054
0,C00006,
1,C00008,T0001
1,C00008,T0002
1,C00008,T0003


In [26]:
# Generate minimal sqlite database from the Amitt variables
conn = sql.connect('amittsite.sqlite')

def add_table(dataframe, tablename, columns):    
    # Create sql table
    colnames = ', '.join(['{} TEXT NOT NULL'.format(col) for col in columns])
    conn.execute("DROP TABLE IF EXISTS {}".format(tablename))
    conn.execute('''CREATE TABLE {} (id INTEGER PRIMARY KEY AUTOINCREMENT, {});'''.format(tablename, colnames))
    #populate table from dataframe
    newtable = dataframe[columns].copy().applymap(str)
    newtable['id'] = range(1,len(newtable)+1)
    newtable.to_sql(tablename, conn, index=False, if_exists='append')
    conn.commit()
    return newtable

def counter_tactics_techniques():
    # techniques - counters
    ctech = amitt.cross_counterid_techniqueid.copy()
    ctech = ctech[(ctech['technique_id'] != '') & (~ctech['technique_id'].str.startswith('TA'))]
    ctech.rename(columns={'amitt_id':'counter_id'}, inplace=True)
    ctech['summary'] = 'N/A'

    # tactics - counters
    ctact = amitt.cross_counterid_techniqueid[amitt.cross_counterid_techniqueid['technique_id'].str.startswith('TA')].copy()
    ctact.rename(columns={'amitt_id':'counter_id', 'technique_id': 'tactic_id'}, inplace=True)
    ctact['main_tactic'] = 'N'
    ctactmain = amitt.df_counters[['amitt_id', 'tactic_id']].copy()
    ctactmain.rename(columns={'amitt_id':'counter_id'}, inplace=True)
    ctactmain['main_tactic'] = 'Y'
    ctact = pd.concat([ctact, ctactmain], ignore_index=True, sort=False)
    ctact['summary'] = 'N/A'
    return(ctech, ctact)

#newtable = add_table(amitt.df_actortypes, 'actor_type', ['amitt_id', 'sector_id', 'framework_id', 'name', 'summary'])
# counter
newtable = add_table(amitt.df_counters, 'counter', ['amitt_id', 'tactic_id', 'metatechnique_id', 'name', 'summary'])
# dataset
newtable = add_table(amitt.df_frameworks, 'framework', ['amitt_id', 'name', 'summary'])
# incident
# metatechnique
newtable = add_table(amitt.df_metatechniques, 'metatechnique', ['amitt_id', 'name', 'summary'])
newtable = add_table(amitt.df_phases, 'phase', ['amitt_id', 'name', 'rank', 'summary'])
# playbook
# reference
# response_type
# sector
newtable = add_table(amitt.df_tactics, 'tactic', ['amitt_id', 'phase_id', 'name', 'rank', 'summary'])
newtable = add_table(amitt.df_tasks, 'task', ['amitt_id', 'tactic_id', 'framework_id', 'name', 'summary'])
newtable = add_table(amitt.df_techniques, 'technique', ['amitt_id', 'tactic_id', 'name', 'summary'])
# technique_counter
(ctech, ctact) = counter_tactics_techniques()
newtable = add_table(ctech, 'counter_technique', ['counter_id', 'technique_id', 'summary'])
newtable = add_table(ctact, 'counter_tactic', ['counter_id', 'tactic_id', 'main_tactic', 'summary'])

conn.execute("DROP TABLE IF EXISTS {}".format('user'))
conn.execute('''CREATE TABLE user (id INTEGER PRIMARY KEY AUTOINCREMENT, username TEXT NOT NULL UNIQUE, password TEXT NOT NULL);''')

conn.close()
newtable

,counter_id,tactic_id,main_tactic,summary,id
0,C00028,TA07,N,N/A,1
1,C00028,TA08,N,N/A,2
2,C00028,TA09,N,N/A,3
3,C00028,TA10,N,N/A,4
4,C00060,TA07,N,N/A,5
5,C00062,TA01,N,N/A,6
6,C00062,TA02,N,N/A,7
7,C00062,TA03,N,N/A,8
8,C00062,TA04,N,N/A,9
9,C00062,TA05,N,N/A,10


In [24]:
ctech

,counter_id,technique_id,summary
0,C00006,T0007,N/A
0,C00006,T0015,N/A
0,C00006,T0018,N/A
0,C00006,T0043,N/A
0,C00006,T0053,N/A
0,C00006,T0054,N/A
1,C00008,T0001,N/A
1,C00008,T0002,N/A
1,C00008,T0003,N/A
1,C00008,T0027,N/A


In [25]:
ctact

,counter_id,tactic_id,main_tactic,summary
0,C00028,TA07,N,N/A
1,C00028,TA08,N,N/A
2,C00028,TA09,N,N/A
3,C00028,TA10,N,N/A
4,C00060,TA07,N,N/A
5,C00062,TA01,N,N/A
6,C00062,TA02,N,N/A
7,C00062,TA03,N,N/A
8,C00062,TA04,N,N/A
9,C00062,TA05,N,N/A
